<h2> Importing Necessary Modules</h2>

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [ ]:
from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss, classification_report, accuracy_score, f1_score

In [ ]:
def file_reader(data):
    return pd.read_csv(data)

In [ ]:
df = file_reader('new_dataset.csv')

In [ ]:
#Checking out the Categorical Column and Numerical in the Dataset
def cat_num(data):
    cat_col = []
    num_col = []
    for feat in data.columns:
        if data[feat].dtypes =='O':
            cat_col.append(feat)
        else:
            num_col.append(feat)
    return cat_col, num_col

In [ ]:
cat_col, num_col = cat_num(train)

In [ ]:
#This columns has a left-skewed KDE plot so using the Numpy Log Transform method increased the model Logloss performance

df.ARPU_SEGMENT = np.log1p(df.ARPU_SEGMENT)
df.DATA_VOLUME = np.log1p(df.DATA_VOLUME)
df.REVENUE = np.log1p(df.REVENUE)
df.MONTANT = np.log1p(df.MONTANT)

In [ ]:
test = df[400000:]

train = df[:400000]

## Preparing the dataset for our ML Model

In [ ]:
X = train[num_col].drop(['CHURN'], axis =1)
y = train.CHURN

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.25, random_state = 43)

In [ ]:
lgb=LGBMClassifier(learning_rate = 0.05,n_estimators=500)

In [ ]:
lgb.fit(X_train,y_train)

In [ ]:
pred = lgb.predict_proba(X_test)[:,1]
pred_x = lgb.predict(X_test)

In [ ]:
f1_score(y_test,pred_x,average='micro')

In [ ]:
log_loss(y_test, pred)  #Logloss of 0.2517 on Public Lb and 0.24722

In [ ]:
_ = plt.figure(figsize = (10,8))
fi = pd.Series(index = X.columns, data = lgm.feature_importances_)
_ = fi.sort_values()[-50:].plot(kind = 'barh')

In [ ]:
sub = file_reader('sample_submission.csv')
sample = sub.copy()
sample.CHURN = lgb_test[:,1]
sample.to_csv('sub_log_transform.csv', index = False)